In [2]:
import pandas as pd
from main import Click_house_cloud
import ast
import nltk
from nltk.stem.porter import PorterStemmer
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
client = Click_house_cloud()

In [4]:
Database = "Create Database IF NOT EXISTS Movies"
Use_db = "Use Movies"
result_db = client.command(Database)
result_use = client.command(Use_db)

In [5]:
query = "SELECT * FROM Movies.Stg_Movies_combined;"

# Execute the query and fetch the data

Movies_Combined = pd.DataFrame(client.query(query).result_rows,columns=["Movie_ID","Title","Overview","Genres","Keywords","Cast","Crew"])


In [6]:
Movies_Combined

,Movie_ID,Title,Overview,Genres,Keywords,Cast,Crew
0,5,Four Rooms,It's Ted the Bellhop's first night on the job....,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name...","[{""id"": 612, ""name"": ""hotel""}, {""id"": 613, ""na...","[{""cast_id"": 42, ""character"": ""Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800012d"", ""de..."
1,11,Star Wars,Princess Leia is captured and held hostage by ...,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...","[{""id"": 803, ""name"": ""android""}, {""id"": 4270, ...","[{""cast_id"": 3, ""character"": ""Luke Skywalker"",...","[{""credit_id"": ""52fe420dc3a36847f8000437"", ""de..."
2,12,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...","[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","[{""id"": 494, ""name"": ""father son relationship""...","[{""cast_id"": 8, ""character"": ""Marlin (voice)"",...","[{""credit_id"": ""52fe420ec3a36847f80006b1"", ""de..."
3,13,Forrest Gump,A man with a low IQ has accomplished great thi...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""...","[{""cast_id"": 7, ""character"": ""Forrest Gump"", ""...","[{""credit_id"": ""52fe420ec3a36847f800076b"", ""de..."
4,14,American Beauty,"Lester Burnham, a depressed suburban father in...","[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 255, ""name"": ""male nudity""}, {""id"": 29...","[{""cast_id"": 6, ""character"": ""Lester Burnham"",...","[{""credit_id"": ""52fe420ec3a36847f8000809"", ""de..."
...,...,...,...,...,...,...,...
4804,426067,Midnight Cabaret,A Broadway producer puts on a play with a Devi...,"[{""id"": 27, ""name"": ""Horror""}]",[],"[{""cast_id"": 1, ""character"": ""Dawn"", ""credit_i...","[{""credit_id"": ""582a4cdb9251417b1803dd47"", ""de..."
4805,426469,Growing Up Smith,"In 1979, an Indian family moves to America wit...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...",[],"[{""cast_id"": 4, ""character"": ""Smith Bhatnagar""...","[{""credit_id"": ""582e0bdfc3a368772600b6c1"", ""de..."
4806,433715,8 Days,"After sneaking to a party with her friends, 16...","[{""id"": 53, ""name"": ""Thriller""}, {""id"": 18, ""n...","[{""id"": 10060, ""name"": ""christian film""}, {""id...","[{""cast_id"": 3, ""character"": ""Amber"", ""credit_...","[{""credit_id"": ""58713acfc3a3684f0a00a9ac"", ""de..."
4807,447027,Running Forever,After being estranged since her mother's death...,"[{""id"": 10751, ""name"": ""Family""}]",[],[],[]


In [7]:
# Analysis Begins
Movies_Combined.isnull().sum()
Movies_Combined.dropna(inplace=True)

In [8]:
Movies_Combined.duplicated().sum()
Movies_Combined.drop_duplicates(inplace=True)

In [9]:
Movies_Combined.head()

,Movie_ID,Title,Overview,Genres,Keywords,Cast,Crew
0,5,Four Rooms,It's Ted the Bellhop's first night on the job....,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name...","[{""id"": 612, ""name"": ""hotel""}, {""id"": 613, ""na...","[{""cast_id"": 42, ""character"": ""Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800012d"", ""de..."
1,11,Star Wars,Princess Leia is captured and held hostage by ...,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...","[{""id"": 803, ""name"": ""android""}, {""id"": 4270, ...","[{""cast_id"": 3, ""character"": ""Luke Skywalker"",...","[{""credit_id"": ""52fe420dc3a36847f8000437"", ""de..."
2,12,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...","[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","[{""id"": 494, ""name"": ""father son relationship""...","[{""cast_id"": 8, ""character"": ""Marlin (voice)"",...","[{""credit_id"": ""52fe420ec3a36847f80006b1"", ""de..."
3,13,Forrest Gump,A man with a low IQ has accomplished great thi...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""...","[{""cast_id"": 7, ""character"": ""Forrest Gump"", ""...","[{""credit_id"": ""52fe420ec3a36847f800076b"", ""de..."
4,14,American Beauty,"Lester Burnham, a depressed suburban father in...","[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 255, ""name"": ""male nudity""}, {""id"": 29...","[{""cast_id"": 6, ""character"": ""Lester Burnham"",...","[{""credit_id"": ""52fe420ec3a36847f8000809"", ""de..."


In [10]:
def transform_text(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [11]:
Movies_Combined['Genres'] = Movies_Combined['Genres'].apply(transform_text)

In [12]:
Movies_Combined

,Movie_ID,Title,Overview,Genres,Keywords,Cast,Crew
0,5,Four Rooms,It's Ted the Bellhop's first night on the job....,"[Crime, Comedy]","[{""id"": 612, ""name"": ""hotel""}, {""id"": 613, ""na...","[{""cast_id"": 42, ""character"": ""Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800012d"", ""de..."
1,11,Star Wars,Princess Leia is captured and held hostage by ...,"[Adventure, Action, Science Fiction]","[{""id"": 803, ""name"": ""android""}, {""id"": 4270, ...","[{""cast_id"": 3, ""character"": ""Luke Skywalker"",...","[{""credit_id"": ""52fe420dc3a36847f8000437"", ""de..."
2,12,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...","[Animation, Family]","[{""id"": 494, ""name"": ""father son relationship""...","[{""cast_id"": 8, ""character"": ""Marlin (voice)"",...","[{""credit_id"": ""52fe420ec3a36847f80006b1"", ""de..."
3,13,Forrest Gump,A man with a low IQ has accomplished great thi...,"[Comedy, Drama, Romance]","[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""...","[{""cast_id"": 7, ""character"": ""Forrest Gump"", ""...","[{""credit_id"": ""52fe420ec3a36847f800076b"", ""de..."
4,14,American Beauty,"Lester Burnham, a depressed suburban father in...",[Drama],"[{""id"": 255, ""name"": ""male nudity""}, {""id"": 29...","[{""cast_id"": 6, ""character"": ""Lester Burnham"",...","[{""credit_id"": ""52fe420ec3a36847f8000809"", ""de..."
...,...,...,...,...,...,...,...
4804,426067,Midnight Cabaret,A Broadway producer puts on a play with a Devi...,[Horror],[],"[{""cast_id"": 1, ""character"": ""Dawn"", ""credit_i...","[{""credit_id"": ""582a4cdb9251417b1803dd47"", ""de..."
4805,426469,Growing Up Smith,"In 1979, an Indian family moves to America wit...","[Comedy, Family, Drama]",[],"[{""cast_id"": 4, ""character"": ""Smith Bhatnagar""...","[{""credit_id"": ""582e0bdfc3a368772600b6c1"", ""de..."
4806,433715,8 Days,"After sneaking to a party with her friends, 16...","[Thriller, Drama]","[{""id"": 10060, ""name"": ""christian film""}, {""id...","[{""cast_id"": 3, ""character"": ""Amber"", ""credit_...","[{""credit_id"": ""58713acfc3a3684f0a00a9ac"", ""de..."
4807,447027,Running Forever,After being estranged since her mother's death...,[Family],[],[],[]


In [13]:
Movies_Combined['Keywords'] = Movies_Combined['Keywords'].apply(transform_text)


In [14]:
def Cast_Transform(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [15]:
Movies_Combined['Cast'] = Movies_Combined['Cast'].apply(Cast_Transform)

In [23]:
Movies_Combined

,Movie_ID,Title,Overview,Genres,Keywords,Cast,Crew,Tags
0,5,Four Rooms,"[It's, Ted, the, Bellhop's, first, night, on, ...","[Crime, Comedy]","[hotel, newyear'seve, witch, bet, hotelroom, s...","[TimRoth, AntonioBanderas, JenniferBeals]",[AllisonAnders],"[It's, Ted, the, Bellhop's, first, night, on, ..."
1,11,Star Wars,"[Princess, Leia, is, captured, and, held, host...","[Adventure, Action, ScienceFiction]","[android, galaxy, hermit, deathstar, lightsabe...","[MarkHamill, HarrisonFord, CarrieFisher]",[GeorgeLucas],"[Princess, Leia, is, captured, and, held, host..."
2,12,Finding Nemo,"[Nemo,, an, adventurous, young, clownfish,, is...","[Animation, Family]","[fathersonrelationship, harbor, underwater, fi...","[AlbertBrooks, EllenDeGeneres, AlexanderGould]",[AndrewStanton],"[Nemo,, an, adventurous, young, clownfish,, is..."
3,13,Forrest Gump,"[A, man, with, a, low, IQ, has, accomplished, ...","[Comedy, Drama, Romance]","[vietnamveteran, hippie, mentallydisabled, run...","[TomHanks, RobinWright, GarySinise]",[RobertZemeckis],"[A, man, with, a, low, IQ, has, accomplished, ..."
4,14,American Beauty,"[Lester, Burnham,, a, depressed, suburban, fat...",[Drama],"[malenudity, femalenudity, adultery, midlifecr...","[KevinSpacey, AnnetteBening, ThoraBirch]",[SamMendes],"[Lester, Burnham,, a, depressed, suburban, fat..."
...,...,...,...,...,...,...,...,...
4804,426067,Midnight Cabaret,"[A, Broadway, producer, puts, on, a, play, wit...",[Horror],[],"[LisaHartCarroll, MichaelDesBarres, PaulDrake]",[PeceDingo],"[A, Broadway, producer, puts, on, a, play, wit..."
4805,426469,Growing Up Smith,"[In, 1979,, an, Indian, family, moves, to, Ame...","[Comedy, Family, Drama]",[],"[RoniAkurati, BrightonSharbino, JasonLee]",[FrankLotito],"[In, 1979,, an, Indian, family, moves, to, Ame..."
4806,433715,8 Days,"[After, sneaking, to, a, party, with, her, fri...","[Thriller, Drama]","[christianfilm, sextrafficking]","[NicoleSmolen, KimBaldwin, ArianaStephens]",[JacoBooyens],"[After, sneaking, to, a, party, with, her, fri..."
4807,447027,Running Forever,"[After, being, estranged, since, her, mother's...",[Family],[],[],[],"[After, being, estranged, since, her, mother's..."


In [16]:
def Crew_Transform(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == "Director":
            L.append(i['name'])
            break
    return L

In [17]:
Movies_Combined['Crew'] = Movies_Combined['Crew'].apply(Crew_Transform)

In [18]:
Movies_Combined['Overview'] = Movies_Combined['Overview'].apply(lambda x: x.split())

In [24]:
Movies_Combined

,Movie_ID,Title,Overview,Genres,Keywords,Cast,Crew,Tags
0,5,Four Rooms,"[It's, Ted, the, Bellhop's, first, night, on, ...","[Crime, Comedy]","[hotel, newyear'seve, witch, bet, hotelroom, s...","[TimRoth, AntonioBanderas, JenniferBeals]",[AllisonAnders],"[It's, Ted, the, Bellhop's, first, night, on, ..."
1,11,Star Wars,"[Princess, Leia, is, captured, and, held, host...","[Adventure, Action, ScienceFiction]","[android, galaxy, hermit, deathstar, lightsabe...","[MarkHamill, HarrisonFord, CarrieFisher]",[GeorgeLucas],"[Princess, Leia, is, captured, and, held, host..."
2,12,Finding Nemo,"[Nemo,, an, adventurous, young, clownfish,, is...","[Animation, Family]","[fathersonrelationship, harbor, underwater, fi...","[AlbertBrooks, EllenDeGeneres, AlexanderGould]",[AndrewStanton],"[Nemo,, an, adventurous, young, clownfish,, is..."
3,13,Forrest Gump,"[A, man, with, a, low, IQ, has, accomplished, ...","[Comedy, Drama, Romance]","[vietnamveteran, hippie, mentallydisabled, run...","[TomHanks, RobinWright, GarySinise]",[RobertZemeckis],"[A, man, with, a, low, IQ, has, accomplished, ..."
4,14,American Beauty,"[Lester, Burnham,, a, depressed, suburban, fat...",[Drama],"[malenudity, femalenudity, adultery, midlifecr...","[KevinSpacey, AnnetteBening, ThoraBirch]",[SamMendes],"[Lester, Burnham,, a, depressed, suburban, fat..."
...,...,...,...,...,...,...,...,...
4804,426067,Midnight Cabaret,"[A, Broadway, producer, puts, on, a, play, wit...",[Horror],[],"[LisaHartCarroll, MichaelDesBarres, PaulDrake]",[PeceDingo],"[A, Broadway, producer, puts, on, a, play, wit..."
4805,426469,Growing Up Smith,"[In, 1979,, an, Indian, family, moves, to, Ame...","[Comedy, Family, Drama]",[],"[RoniAkurati, BrightonSharbino, JasonLee]",[FrankLotito],"[In, 1979,, an, Indian, family, moves, to, Ame..."
4806,433715,8 Days,"[After, sneaking, to, a, party, with, her, fri...","[Thriller, Drama]","[christianfilm, sextrafficking]","[NicoleSmolen, KimBaldwin, ArianaStephens]",[JacoBooyens],"[After, sneaking, to, a, party, with, her, fri..."
4807,447027,Running Forever,"[After, being, estranged, since, her, mother's...",[Family],[],[],[],"[After, being, estranged, since, her, mother's..."


In [19]:
Movies_Combined['Overview'] = Movies_Combined['Overview'].apply(lambda x:[i.replace(" ","") for i in x])
Movies_Combined['Genres'] = Movies_Combined['Genres'].apply(lambda x:[i.replace(" ","") for i in x])
Movies_Combined['Keywords'] = Movies_Combined['Keywords'].apply(lambda x:[i.replace(" ","") for i in x])
Movies_Combined['Cast'] = Movies_Combined['Cast'].apply(lambda x:[i.replace(" ","") for i in x])
Movies_Combined['Crew'] = Movies_Combined['Crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [20]:
Movies_Combined['Tags'] = Movies_Combined['Overview'] + Movies_Combined['Genres'] + Movies_Combined['Keywords'] + Movies_Combined['Cast'] + Movies_Combined['Crew']

In [21]:
Movies_Combined.head(10)

,Movie_ID,Title,Overview,Genres,Keywords,Cast,Crew,Tags
0,5,Four Rooms,"[It's, Ted, the, Bellhop's, first, night, on, ...","[Crime, Comedy]","[hotel, newyear'seve, witch, bet, hotelroom, s...","[TimRoth, AntonioBanderas, JenniferBeals]",[AllisonAnders],"[It's, Ted, the, Bellhop's, first, night, on, ..."
1,11,Star Wars,"[Princess, Leia, is, captured, and, held, host...","[Adventure, Action, ScienceFiction]","[android, galaxy, hermit, deathstar, lightsabe...","[MarkHamill, HarrisonFord, CarrieFisher]",[GeorgeLucas],"[Princess, Leia, is, captured, and, held, host..."
2,12,Finding Nemo,"[Nemo,, an, adventurous, young, clownfish,, is...","[Animation, Family]","[fathersonrelationship, harbor, underwater, fi...","[AlbertBrooks, EllenDeGeneres, AlexanderGould]",[AndrewStanton],"[Nemo,, an, adventurous, young, clownfish,, is..."
3,13,Forrest Gump,"[A, man, with, a, low, IQ, has, accomplished, ...","[Comedy, Drama, Romance]","[vietnamveteran, hippie, mentallydisabled, run...","[TomHanks, RobinWright, GarySinise]",[RobertZemeckis],"[A, man, with, a, low, IQ, has, accomplished, ..."
4,14,American Beauty,"[Lester, Burnham,, a, depressed, suburban, fat...",[Drama],"[malenudity, femalenudity, adultery, midlifecr...","[KevinSpacey, AnnetteBening, ThoraBirch]",[SamMendes],"[Lester, Burnham,, a, depressed, suburban, fat..."
5,16,Dancer in the Dark,"[Selma,, a, Czech, immigrant, on, the, verge, ...","[Drama, Crime, Music]","[individual, dancing, usa, robbery, factorywor...","[Björk, CatherineDeneuve, DavidMorse]",[LarsvonTrier],"[Selma,, a, Czech, immigrant, on, the, verge, ..."
6,18,The Fifth Element,"[In, 2257,, a, taxi, driver, is, unintentional...","[Adventure, Fantasy, Action, Thriller, Science...","[clone, taxi, cyborg, egypt, future, stowaway,...","[BruceWillis, GaryOldman, IanHolm]",[LucBesson],"[In, 2257,, a, taxi, driver, is, unintentional..."
7,19,Metropolis,"[In, a, futuristic, city, sharply, divided, be...","[Drama, ScienceFiction]","[manvsmachine, undergroundworld, inventor, met...","[BrigitteHelm, AlfredAbel, GustavFröhlich]",[FritzLang],"[In, a, futuristic, city, sharply, divided, be..."
8,20,My Life Without Me,"[A, Pedro, Almodovar, production, in, which, a...","[Drama, Romance]","[farewell, responsability, dyinganddeath, nigh...","[SarahPolley, AmandaPlummer, ScottSpeedman]",[IsabelCoixet],"[A, Pedro, Almodovar, production, in, which, a..."
9,22,Pirates of the Caribbean: The Curse of the Bla...,"[Jack, Sparrow,, a, freewheeling, 17th-century...","[Adventure, Fantasy, Action]","[exoticisland, blacksmith, eastindiatradingcom...","[JohnnyDepp, GeoffreyRush, OrlandoBloom]",[GoreVerbinski],"[Jack, Sparrow,, a, freewheeling, 17th-century..."


In [22]:
Movies_Combined.head(2)

,Movie_ID,Title,Overview,Genres,Keywords,Cast,Crew,Tags
0,5,Four Rooms,"[It's, Ted, the, Bellhop's, first, night, on, ...","[Crime, Comedy]","[hotel, newyear'seve, witch, bet, hotelroom, s...","[TimRoth, AntonioBanderas, JenniferBeals]",[AllisonAnders],"[It's, Ted, the, Bellhop's, first, night, on, ..."
1,11,Star Wars,"[Princess, Leia, is, captured, and, held, host...","[Adventure, Action, ScienceFiction]","[android, galaxy, hermit, deathstar, lightsabe...","[MarkHamill, HarrisonFord, CarrieFisher]",[GeorgeLucas],"[Princess, Leia, is, captured, and, held, host..."


In [21]:
Movies_Combined_DF = Movies_Combined[['Movie_ID','Title','Tags']]

In [22]:
Movies_Combined_DF.head()

,Movie_ID,Title,Tags
0,5,Four Rooms,"[It's, Ted, the, Bellhop's, first, night, on, ..."
1,11,Star Wars,"[Princess, Leia, is, captured, and, held, host..."
2,12,Finding Nemo,"[Nemo,, an, adventurous, young, clownfish,, is..."
3,13,Forrest Gump,"[A, man, with, a, low, IQ, has, accomplished, ..."
4,14,American Beauty,"[Lester, Burnham,, a, depressed, suburban, fat..."


In [23]:
Movies_Combined_DF['Tags'] = Movies_Combined_DF['Tags'].apply(lambda x: " ".join(x))

C:\Users\Admin\AppData\Local\Temp\ipykernel_16592\4075441077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Movies_Combined_DF['Tags'] = Movies_Combined_DF['Tags'].apply(lambda x: " ".join(x))


In [24]:
Movies_Combined_DF['Tags'] = Movies_Combined_DF['Tags'].apply(lambda x: x.lower())

C:\Users\Admin\AppData\Local\Temp\ipykernel_16592\408037973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Movies_Combined_DF['Tags'] = Movies_Combined_DF['Tags'].apply(lambda x: x.lower())


In [25]:
Movies_Combined_DF.head()

,Movie_ID,Title,Tags
0,5,Four Rooms,it's ted the bellhop's first night on the job....
1,11,Star Wars,princess leia is captured and held hostage by ...
2,12,Finding Nemo,"nemo, an adventurous young clownfish, is unexp..."
3,13,Forrest Gump,a man with a low iq has accomplished great thi...
4,14,American Beauty,"lester burnham, a depressed suburban father in..."


In [26]:
Movies_Combined_DF['Movie_ID'] = Movies_Combined_DF['Movie_ID'].astype('object')

C:\Users\Admin\AppData\Local\Temp\ipykernel_16592\2425005121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Movies_Combined_DF['Movie_ID'] = Movies_Combined_DF['Movie_ID'].astype('object')


In [31]:
Movies_Combined_DF.dtypes

Movie_ID    object
Title       object
Tags        object
dtype: object

In [32]:
result_tab_Credit = client.command(
    'CREATE TABLE IF NOT EXISTS Movies.Final (Movie_ID Int ,Title String,Tags String)ENGINE MergeTree order by Movie_ID;')

In [33]:
data = Movies_Combined_DF.values.tolist()  # Convert DataFrame to list of lists
column_names = list(Movies_Combined_DF.columns)  # Get column names

client.insert('Movies.Final', data, column_names=column_names)

print("Data inserted successfully into ClickHouse")

Data inserted successfully into ClickHouse
